# Amplitude model with `sympy`

In this notebook, we formulate the amplitude model for the $\gamma p \to K^+ \pi^0 \Lambda $ symbolically by adapting the model originally for the $\gamma p \to \eta\pi^0 p$ channel example as described in [Reaction and Models](reaction-model.md).

The model we want to implement is

$$
\begin{array}{rcl}
I &=& \left|A^{12} + A^{23} + A^{31}\right|^2 \\
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{K^{*+}_2}+im_{K^{*+}_2} \Gamma_{K^{*+}_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Sigma^*}+im_{\Sigma^*} \Gamma_{\Sigma^*}} \\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^{*+}}+im_{N^{*+}} \Gamma_{N^{*+}}} \,,
\end{array}
$$

where $1=K^+$, $2=\pi^0$, and $3=\Lambda$.

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

import sympy as sp

STATIC_PAGE = "EXECUTE_NB" in os.environ

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

## Model implementation

In [ ]:
l_max = 2

### $A^{12}$

In [ ]:
s12, m_Kstar2, Gamma_Kstar2, l12 = sp.symbols(r"s_{12} m_{K^*_2} \Gamma_{K^*_2} l_{12}")
theta1, phi1 = sp.symbols("theta_1 phi_1")
a = sp.IndexedBase("a")
m = sp.symbols("m", cls=sp.Idx)
A12 = sp.Sum(a[m] * sp.Ynm(l12, m, theta1, phi1), (m, -l12, l12)) / (
    s12 - m_Kstar2**2 + sp.I * m_Kstar2 * Gamma_Kstar2
)
A12

In [ ]:
A12_funcs = [
    sp.lambdify(
        [
            s12,
            *(a[j] for j in range(-l_max, l_max + 1)),
            m_Kstar2,
            Gamma_Kstar2,
            theta1,
            phi1,
        ],
        expr=A12.subs(l12, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A12_funcs

### $A^{23}$

In [ ]:
s23, m_Sigma, Gamma_Sigma, l23 = sp.symbols(
    r"s_{23} m_{\Sigma^{*+}} \Gamma_{\Sigma^{*+}} l_{23}"
)
b = sp.IndexedBase("b")
m = sp.symbols("m", cls=sp.Idx)
theta2, phi2 = sp.symbols("theta_2 phi_2")
A23 = sp.Sum(b[m] * sp.Ynm(l23, m, theta2, phi2), (m, -l23, l23)) / (
    s23 - m_Sigma**2 + sp.I * m_Sigma * Gamma_Sigma
)
A23

In [ ]:
A23_funcs = [
    sp.lambdify(
        [
            s23,
            *(b[j] for j in range(-l_max, l_max + 1)),
            m_Sigma,
            Gamma_Sigma,
            theta2,
            phi2,
        ],
        A23.subs(l23, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A23_funcs

### $A^{31}$

In [ ]:
s31, m_Nstar, Gamma_Nstar = sp.symbols(r"s_{31} m_{N^*} \Gamma_{N^*}")
c = sp.IndexedBase("c")
theta3, phi3, l31 = sp.symbols("theta_3 phi_3 l_{31}")
A31 = sp.Sum(c[m] * sp.Ynm(l31, m, theta3, phi3), (m, -l31, l31)) / (
    s31 - m_Nstar**2 + sp.I * m_Nstar * Gamma_Nstar
)
A31

In [ ]:
A31_funcs = [
    sp.lambdify(
        [
            s31,
            *(c[j] for j in range(-l_max, l_max + 1)),
            m_Nstar,
            Gamma_Nstar,
            theta3,
            phi3,
        ],
        A31.subs(l31, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A31_funcs

### $I = |A|^2 = |A^{12}+A^{23}+A^{31}|^2$

In [ ]:
intensity_expr = sp.Abs(A12 + A23 + A31) ** 2
intensity_expr